# Titanic Problem Using Pandas
## Simple Approach to form a Survivor Table

In [26]:
import pandas as pd
import numpy as np
import csv

In [27]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [28]:
ceil_fare = 40
fair_bracket_size = 10
gender_count = len(df['Sex'].unique())
classes = len(df['Pclass'].unique())
df.loc[(df['Fare'] > 40),'Fare'] = ceil_fare - 1
price_brackets = ceil_fare/fair_bracket_size

In [29]:
survivor_table = np.zeros((gender_count,classes,price_brackets))
np.shape(survivor_table)

(2, 3, 4)

In [30]:
df[['Survived','Pclass','Sex','Fare']].head()

,Survived,Pclass,Sex,Fare
0,0,3,male,7.250
1,1,1,female,39.000
2,1,3,female,7.925
3,1,1,female,39.000
4,0,3,male,8.050


In [31]:
for i in xrange(classes):
    for j in xrange(price_brackets):
        survivor_table[0,i,j] = df.loc[(df['Sex'] == 'female') & (df['Fare'] >= j*fair_bracket_size) & (df['Fare'] < (j+1)*fair_bracket_size) & (df['Pclass'] == i+1),'Survived'].mean()
        survivor_table[1,i,j] = df.loc[(df['Sex'] == 'male') & (df['Fare'] >= j*fair_bracket_size) & (df['Fare'] <= (j+1)*fair_bracket_size) & (df['Pclass'] == i+1),'Survived'].mean()

In [32]:
survivor_table[survivor_table != survivor_table] = 0
survivor_table[survivor_table >= 0.5] = 1
survivor_table[survivor_table < 0.5] = 0
print survivor_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [40]:
file_handle = open('test.csv','r')
prediction_handle = open('output3.csv','w')
fp = csv.reader(file_handle)
p = csv.writer(prediction_handle)
header = fp.next()
p.writerow(['PassengerId','Survived'])

In [44]:
for row in fp:
    for j in xrange(price_brackets):
        try:
            row[8] = float(row[8])
        except:
            bin_fare = 3 - float(row[1])
            break
        if(row[8] > ceil_fare):
            bin_fare = price_brackets - 1
            break
        if(row[8] >= j*fair_bracket_size and row[8] < (j+1)*fair_bracket_size):
            bin_fare = j
            break
    if(row[3] == 'female'):
        p.writerow([row[0],"%d"%int(survivor_table[0,float(row[1]) - 1,bin_fare])])
    else:
        p.writerow([row[0],"%d"%int(survivor_table[1, float(row[1])-1, bin_fare])])
file_handle.close()
prediction_handle.close()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
